In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime as dt
import statistics as stats
import numpy as np
import yfinance as yf
import get_tickers as gt
from get_tickers import *
from scipy.stats import norm



In [2]:

df_health = pd.read_csv('tick_score_health.csv')
df_tech = pd.read_csv('tick_score_tech.csv')
df_durable = pd.read_csv('tick_score_durable.csv')
df_nondurable = pd.read_csv('tick_score_nondurable.csv')
df_capital = pd.read_csv('tick_score_capital.csv')
df_transport = pd.read_csv('tick_score_transport.csv')
df_util = pd.read_csv('tick_score_util.csv')
df_services = pd.read_csv('tick_score_services.csv')
df_energy = pd.read_csv('tick_score_energy.csv')
df_finance = pd.read_csv('tick_score_finance.csv')
df_basics = pd.read_csv('tick_score_basics.csv')

In [3]:
frames = [df_finance, df_capital, df_durable, df_energy, df_health, df_nondurable, df_services, df_tech, df_services, df_util, df_basics]
df = pd.concat(frames)
df = df.drop_duplicates(subset=['Unnamed: 0'], keep='last')
df = df.set_index("Unnamed: 0")

percentile = np.percentile(df['Score'], 90)
mu, std = norm.fit(df['Score'])
roc = std / 3
aplus = df[df['Score'] > percentile - (roc*2)]

tickers = df.index
print(tickers)

Index(['MCO', 'SPGI', 'TRV', 'BEN', 'GS', 'BLK', 'AON', 'AMP', 'KKR', 'LPLA',
       ...
       'AES', 'WST', 'FBHS', 'WPM', 'PKX', 'NEM', 'J', 'AEM', 'NUE', 'TS'],
      dtype='object', name='Unnamed: 0', length=564)


In [4]:
def SharpeRatio(rt_df, bncmrk=0.02):
    SR = np.sqrt(len(rt_df))*(rt_df['NetWorth'].mean() - bncmrk) / rt_df['NetWorth'].std()
    return SR

def get_metrics(df, digits=3, periods = 0):
    risk_free_return = 0
    df['return'] = df['NetWorth'] / df['NetWorth'][0]
    max_drawdown = 1.0 - min(df['return'].values)
    df['return_pct'] = df['NetWorth'].pct_change(1)
    sharpe = (df['NetWorth'].mean() - risk_free_return) / df['NetWorth'].std()
    sharpe *= periods ** 0.5 #annualize
    sharpe /= 100
    #sharpe = np.mean(ret) / np.std(ret)
    return round(df['NetWorth'][-1]/df['NetWorth'][0], digits), round(sharpe, digits), round(max_drawdown * 100, digits-2)

In [7]:
df_metrics = []
df_metrics = pd.DataFrame(index=tickers, columns=['MACD_12_26', 'MACD_50_200', 'William_R', 'BB_10', 'BB_21', 'BB_50', 'RSI', 'Stoch_Osc', 'VWAP', 'ATR_2', 'ATR_10', 'ATR_30', 'ATR_50'])

#df_results = df_results.set_index('ticker')
#df_results[['MACD','William_R','Boll_Bands', 'RSI', 'stoch_osc', 'VWAP']]        
#start = dt.date.today() - dt.timedelta(days = 365)
#end = dt.date.today()
start = "2016-01-01"
end = "2021-01-01"
df_results = []


In [9]:

def add_metric(log, periods = 0):
    log_df = pd.DataFrame(log, columns = ['Date', 'Price', 'StockAmount', 'Buy/Sell', 'NetWorth'])
    log_df = log_df.set_index('Date')
    return get_metrics(log_df, 3, periods)

def RSI(df):
    df['sumGain'] = 0
    df['sumLoss'] = 0
    
    for i in range(1, len(df)-1):
        difference = df['Adj Close'][i] - df['Adj Close'][i-1]
        
        if difference > 0:
            df['sumGain'][i] += difference
            df['sumLoss'][i] = df['sumLoss'][i-1]
        elif difference < 0: 
            df['sumLoss'][i] -= difference
            df['sumGain'][i] = df['sumGain'][i-1]
        elif difference == 0:
            df['sumGain'][i] = df['sumGain'][i-1]
            df['sumLoss'][i] = df['sumLoss'][i-1]
        
    
    rs = pd.Series.ewm(df['sumGain'], span=30).mean()/(pd.Series.ewm(df['sumLoss'], span=30).mean() + pd.Series.ewm(df['sumGain'], span=30).mean())
    df = df.assign(RSI = pd.Series(rs))
    return df

def william_R(df):
    df['wr'] = 0
    hh = df['High'].rolling(30).max() #highest high over lookback period lbp
    ll = df['Low'].rolling(30).min()  #lowest low over lookback period lbp
    close = df['Close']    
    wr = (-100 * (hh - close) / (hh - ll))
    wr = wr.fillna(-50)
    df['wr'] = wr


    budget = 1000
    net = []
    stock_amt = 0
    
    for i in range(0, len(df)-1):
        if (df['wr'][i] >= -20) and (stock_amt != 0):
            budget = stock_amt * df["Close"][i]
            stock_amt = 0
            net.append([df.index[i], df["Close"][i], stock_amt, 0, budget])
        if df['wr'][i] <= -80 and (budget != 0):
            stock_amt = float(budget / df["Close"][i])
            budget = 0
            net.append([df.index[i], df["Close"][i], stock_amt, 1, stock_amt * df["Close"][i]])
            
        
    return df, net

def RSI_new(prices, dates, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)
    budget = 1000
    net = []
    stock_amt = 0
    
    for i in range(n, len(prices)):
        delta = deltas[i-1]  # The diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    df_rsi = pd.concat([pd.Series(dates), pd.Series(rsi)], axis = 1)
    df_rsi.set_index('Date', inplace=True)
    df_rsi.columns = ['RSI']
    for i in range(0, len(df_rsi)-1):
        if df_rsi['RSI'][i] > 70 and (stock_amt != 0):
            budget = stock_amt * prices[i]
            stock_amt = 0
            net.append([df_rsi.index[i], prices[i], stock_amt, 0, budget])
        if df_rsi['RSI'][i] < 30 and (budget != 0):
            stock_amt = float(budget / prices[i])
            budget = 0
            net.append([df_rsi.index[i], prices[i], stock_amt, 1, stock_amt * prices[i]])
    return df_rsi, net

def ATR(df, days = 20):
    df['BUY_Signal'] = 0
    df['SELL_Signal'] = 0
    df['H-L'] = abs(df['High']-df['Low'])
    df['H-PC'] = abs(df['High']-df['Close'].shift(1))
    df['L-PC'] = abs(df['Low']-df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(days).mean()
    df['BUY_Signal'] = (df['TR']*2) + df['Close']
    df['SELL_Signal'] = df['Close'] - (df['TR']*2)
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L', 'H-PC', 'L-PC'], axis=1)
    budget = 1000
    net = []
    stock_amt = 0
    
    for i in range(1, len(df)-1):
        # An upside breakout occurs when the price goes 1 ATR above the previous close
        if df["Close"][i] > df["Close"][i-1] + df['ATR'][i] and (budget != 0):
            stock_amt = float(budget / df["Close"][i])
            budget = 0
            net.append([df.index[i], df["Close"][i], stock_amt, 1, stock_amt * df["Close"][i]])

        # A downside breakout occurs when the previous close is 1 ATR above the price
        if df["Close"][i] < df["Close"][i-1] - df['ATR'][i] and (stock_amt != 0):
            budget = stock_amt * df['Close'][i]
            stock_amt = 0
            net.append([df.index[i], df['Close'][i], stock_amt, 0, budget])
    
    return df2, net

def boll_bands(df, timelen = 21):
    cols = df.columns.values
    df = df.assign(MA = df['Adj Close'].rolling(window=timelen).mean())
    df = df.assign(Upper = df['MA'] + math.sqrt(math.pi))
    df = df.assign(Lower = df['MA'] - math.sqrt(math.pi))
    net = []
    u = 0.03
    l = 0.03
    budget = 1000
    stock_amt = 0

    # Buy
    for i in range(0, len(df)-1):
        if (df['Close'][i] < df['MA'][i]) and (abs(1 - df['Close'][i] / df['Lower'][i]) < u):
            if budget > 0:
                stock_amt = float(budget / df['Close'][i])
                budget = 0
                net.append([df.index[i], df['Close'][i], stock_amt, 1, stock_amt * df['Close'][i]])

        # Sell        
        if (df['Close'][i] > df['MA'][i]) and (abs(1 - df['Upper'][i] / df['Close'][i]) < l):
            if budget == 0:
                budget = stock_amt * df['Close'][i]
                stock_amt = 0
                net.append([df.index[i], df['Close'][i], stock_amt, 0, budget])
            
    df = df.drop(columns=cols)
    return df, net

def ema(df, lowlen = 12, uplen = 26, malen = 15, input = 'Adj Close'):
    cols = df.columns.values
    df = df.assign(MA = df[input].rolling(window=malen).mean())
    df = df.assign(Upper_EMA = pd.Series.ewm(df[input], adjust=False, span=uplen).mean())
    df = df.assign(Lower_EMA = pd.Series.ewm(df[input], adjust=False, span=lowlen).mean())
    df = df.assign(Price = (df['Close']))
    df = df.drop(columns=cols)
    return df
    
def momentum(df, period = 30):
    cols = df.columns.values
    df = df.assign(Momentum = pd.Series(df['Adj Close'].diff(30)))
    df = df.drop(columns=cols)
    return df

def macd(df, lowlen = 12, uplen = 26, malen = 15, macdlen = 9):
    state_check = lambda x, y: 1 if x > y else 0
    df = ema(df, lowlen, uplen, malen)
    df = df.assign(MACD = pd.Series(df["Upper_EMA"] - df["Lower_EMA"]))
    df = df.assign(MACD_Signal = pd.Series.ewm(df['MACD'], adjust=False, span=macdlen).mean())

    
    budget = 1000
    net = []
    stock_amt = 0
    state_check = lambda x, y: 1 if x > y else 0
    
    for i in range(1, len(df)-1):
        state = state_check(df["MACD"][i], df["MACD_Signal"][i])
        prev_state = state_check(df["MACD"][i-1], df["MACD_Signal"][i-1])
        if state != prev_state:
            if((df["MACD"][i-1] <= df["MACD_Signal"][i-1]) and (df["MACD"][i]>df["MACD_Signal"][i])) and (budget != 0):
                stock_amt = float(budget / df["Price"][i])
                budget = 0
                net.append([df.index[i], df["Price"][i], stock_amt, 1, stock_amt * df["Price"][i]])
            
            if((df["MACD"][i-1] >= df["MACD_Signal"][i-1]) and (df["MACD"][i]<df["MACD_Signal"][i])) and (stock_amt != 0):
                budget = stock_amt * df["Price"][i]
                stock_amt = 0
                net.append([df.index[i], df["Price"][i], stock_amt, 0, budget])
    return df, net

def vwap(df):
    cols = df.columns.values
    budget = 1000
    net = []
    stock_amt = 0

    df = df.assign(Price = (df['High'] + df['Low'] + df['Close'])/3)
    df = df.assign(VWAP = (np.cumsum(df['Price'] * df['Volume'])) / np.cumsum(df['Volume']))
    for i in range(0, len(df)-1):
        if df['Close'][i] > (df["VWAP"][i] * 1.005) and (budget != 0):
            stock_amt = float(budget / df["Price"][i])
            budget = 0
            net.append([df.index[i], df["Price"][i], stock_amt, 1, stock_amt * df["Price"][i]])
        if df['Close'][i] < (df["VWAP"][i] * 0.995) and (stock_amt != 0):
            budget = stock_amt * df["Price"][i]
            stock_amt = 0
            net.append([df.index[i], df["Price"][i], stock_amt, 0, budget])
    df = df.drop(columns=cols)
    return df, net

def stochastic_osc(df, lw, sw):
    cols = df.columns.values
    smin = df['Low'].rolling(lw).min()
    smax = df['High'].rolling(lw).max()
    stoch_k = 100 * (df['Close'] - smin) / (smax - smin)
    stoch_k = stoch_k.fillna(50)
    stoch_d = stoch_k.rolling(sw).mean()
    stoch_d = stoch_d.fillna(50)
    budget = 1000
    net = []
    stock_amt = 0

    df = df.assign(Stoch = stoch_k)
    df = df.assign(Stoch_Signal = stoch_d)
    d_osc = derivative_osc(df)
    
    for i in range(0, len(df)-1):    
        if df['Stoch_Signal'][i] <= 20 and (budget != 0):
            stock_amt = float(budget / df["Close"][i])
            budget = 0
            net.append([df.index[i], df["Close"][i], stock_amt, 1, stock_amt * df["Close"][i]])
        if df['Stoch_Signal'][i] >= 80 and (stock_amt != 0):
            budget = stock_amt * df["Close"][i]
            stock_amt = 0
            net.append([df.index[i], df["Close"][i], stock_amt, 0, budget])
    df = df.drop(columns=cols)

    return df, net

def derivative_osc(df):
    cols = df.columns.values
    rsi_df, log = RSI_new(df['Close'], df.index, 14)
    df = df.assign(sm_rsi = pd.Series.ewm(rsi_df['RSI'], adjust=False, span=14).mean())
    df = df.assign(dsm_rsi = pd.Series.ewm(df['sm_rsi'], adjust=False, span=14).mean())
    df = df.assign(sma_rsi = df['dsm_rsi'].rolling(window=14).mean())
    df = df.assign(Signal_Line = df['sma_rsi'] - df['dsm_rsi'])
    df = df.drop(columns=cols)
    return df

for ticker in tickers:
    ticker = ticker.rstrip("\n")
    ticker = ticker.rstrip(" ")
    ticker = ticker.rstrip(" ")
    t = yf.Ticker(ticker)

    #Info
    try:
        row_pct = []
        stock_df = yf.download(ticker,start,end)
        macd_df, log = macd(stock_df, 12, 26, 15, 9)
        macd_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(macd_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["MACD_12_26"] = [profit, sharpe, drawdown]
        
        
        stock_df = yf.download(ticker,start,end)
        macd_df, log = macd(stock_df, 50, 200, 50, 26)
        macd_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(macd_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["MACD_50_200"] = [profit, sharpe, drawdown]
        
        #William %R
        wr_df, log = william_R(stock_df)
        wr_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(wr_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["William_R"] = [profit, sharpe, drawdown]
        
        #Bollinger Bands
        boll_df, log = boll_bands(stock_df, 10)
        boll_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(boll_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["BB_10"] = [profit, sharpe, drawdown]
        
        boll_df, log = boll_bands(stock_df, 21)
        boll_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(boll_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["BB_21"] = [profit, sharpe, drawdown]
        
        boll_df, log = boll_bands(stock_df, 50)
        boll_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(boll_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["BB_50"] = [profit, sharpe, drawdown]
        
        stoch_df, log = stochastic_osc(stock_df, 14, 3)
        stoch_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(stoch_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["Stoch_Osc"] = [profit, sharpe, drawdown]
        
        #Volume Weighted Adjusted Price
        vwap_df, log = vwap(stock_df)
        vwap_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(vwap_pct)
        df_results.append([ticker, row_pct])
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["VWAP"] = [profit, sharpe, drawdown]
        
        #Relative Strength Index
        rsi_df, log = RSI_new(stock_df['Close'], stock_df.index, 14)
        rsi_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(rsi_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["RSI"] = [profit, sharpe, drawdown]
        
        
        atr_df, log = ATR(stock_df, 30)
        atr_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(atr_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["ATR_30"] = [profit, sharpe, drawdown]
        
        atr_df, log = ATR(stock_df, 10)
        atr_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(atr_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["ATR_10"] = [profit, sharpe, drawdown]
        
        
        atr_df, log = ATR(stock_df, 2)
        atr_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(atr_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["ATR_2"] = [profit, sharpe, drawdown]
        
        atr_df, log = ATR(stock_df, 50)
        atr_pct = (((log[-1][-1]-1000) / 1000)*100)
        row_pct.append(atr_pct)
        profit, sharpe, drawdown = add_metric(log, len(macd_df))
        df_metrics.loc[ticker]["ATR_50"] = [profit, sharpe, drawdown]
    except:
        pass

metrics_stats = pd.DataFrame(index=['Mean Profit Factor', 'Mean Sharpe Ratio', 'Mean Max Drawdown'], columns=['MACD_12_26', 'MACD_50_200', 'William_R', 'BB_10', 'BB_21', 'BB_50', 'RSI', 'Stoch_Osc', 'VWAP', 'ATR_2', 'ATR_10', 'ATR_30', 'ATR_50'])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
for col in df_metrics.columns:
    lst_lst = df_metrics[col]
    profit = [item[0] for item in lst_lst if item == item]
    sharpe = [item[1] for item in lst_lst if item == item]
    sharpe = [item for item in sharpe if item == item]
    max_drawdown = [item[2] for item in lst_lst if item == item]
    metrics_stats.loc["Mean Profit Factor"][col] = np.mean(profit)
    metrics_stats.loc["Mean Sharpe Ratio"][col] = np.mean(sharpe)
    metrics_stats.loc["Mean Max Drawdown"][col] = np.mean(max_drawdown)


print(metrics_stats)
metrics_stats.to_csv("bull_metrics_stats_all_a.csv")

                   MACD_12_26 MACD_50_200 William_R    BB_10    BB_21  \
Mean Profit Factor    1.40009     1.44801   1.45872  1.30444  1.31134   
Mean Sharpe Ratio     2.30807     2.62926   2.60499  4.64224  3.73527   
Mean Max Drawdown     20.4719     11.1187   10.4561  15.1933  14.0548   

                      BB_50      RSI Stoch_Osc     VWAP    ATR_2   ATR_10  \
Mean Profit Factor  1.27634      NaN   1.51098  1.11252  1.52872  1.47704   
Mean Sharpe Ratio   4.20367  2.94301   2.65049   6.4265  2.77123  2.64937   
Mean Max Drawdown   10.1303      NaN   10.8792  9.98502  14.4205  15.0951   

                     ATR_30   ATR_50  
Mean Profit Factor  1.47673  1.41004  
Mean Sharpe Ratio   2.54464  2.57678  
Mean Max Drawdown   16.7788  18.8713  
